In [3]:
from datasets import load_dataset
from PIL import Image
import io
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

dataset = load_dataset("/home/huayu/pickapic_v2",split="validation_unique")

Resolving data files: 100%|██████████| 645/645 [00:00<00:00, 358987.01it/s]


In [4]:
# Can do clip_utils, aes_utils, hps_utils
from utils.pickscore_utils import Selector
# Score generations automatically w/ reward model
ps_selector = Selector('cuda')

`text_config_dict` is provided which will be used to initialize `CLIPTextConfig`. The value `text_config["id2label"]` will be overriden.


In [5]:
imageN= 500
prompts = dataset[:imageN]['caption']
prefered_imgs = []
disprefered_imgs = []
for i in range(imageN):
    jpg_0 = Image.open(io.BytesIO(dataset[i]['jpg_0'])).convert("RGB")
    jpg_1 = Image.open(io.BytesIO(dataset[i]['jpg_1'])).convert("RGB")
    prefered = jpg_0 if dataset[i]['label_0'] > 0.5 else jpg_1
    disprefered = jpg_1 if dataset[i]['label_0'] > 0.5 else jpg_0
    prefered_imgs.append(prefered)
    disprefered_imgs.append(disprefered)


In [6]:
from diffusers import StableDiffusionPipeline, UNet2DConditionModel, StableDiffusionXLPipeline
import torch
# pretrained_model_name = "CompVis/stable-diffusion-v1-4"
pretrained_model_name = "/home/huayu/stable-diffusion-v1-5"
# pretrained_model_name = "stabilityai/stable-diffusion-xl-base-1.0"
gs = (5 if 'stable-diffusion-xl' in pretrained_model_name else 7.5)
torch.set_grad_enabled(False)
if 'stable-diffusion-xl' in pretrained_model_name:
    pipe = StableDiffusionXLPipeline.from_pretrained(
        pretrained_model_name, torch_dtype=torch.float16,
        variant="fp16", use_safetensors=True
    ).to("cuda")
else:
    pipe = StableDiffusionPipeline.from_pretrained(pretrained_model_name,
                                                   torch_dtype=torch.float16)
pipe = pipe.to('cuda')
pipe.safety_checker = None # Trigger-happy, blacks out >50% of "robot tiger"


baseline_images = []
for i in range(3):
    generator = torch.Generator(device='cuda')
    generator = generator.manual_seed(0)
    img = pipe(prompt=prompts[i:(i+1)], generator=generator, guidance_scale=gs).images
    baseline_images += img

Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  3.86it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 39.41 GiB total capacity; 6.23 GiB already allocated; 29.56 MiB free; 6.28 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

: 

In [12]:
len(prompts)

500

In [ ]:
scores = ps_selector.score(prompts, prefered_imgs)